In [1]:
import geopandas as gpd
import pandas as ps

## immprovement polygons

In [47]:
justImprovement = gpd.read_file('times/newTimesImprov.shp')
justImprovement = justImprovement.to_crs(epsg=4326)
justImprovement.drop(['newTimeMin','plata','secStd','tiemposMin','tiemposSec','x','y','diffP'],axis=1,inplace=True)
justImprovement.columns = ['CO_FRAC_RA','AhorroMinutos','geometry',] 
justImprovement = justImprovement.query('AhorroMinutos <0')
justImprovement['popupContent'] = justImprovement.AhorroMinutos.map(lambda x: "Se ahorran en total %02d:%02d minutos al centro" % (abs(int(x)),abs((x-int(x)))*60))
#justImprovement.drop(['AhorroMinutos'],axis=1,inplace=True)
justImprovement.to_file('../improvementRadios', driver="GeoJSON")
justImprovement.head()

,CO_FRAC_RA,AhorroMinutos,geometry,popupContent
3,10_12_9,-2.299146,POLYGON ((-58.48213672307894 -34.6320075355014...,Se ahorran en total 02:17 minutos al centro
5,10_13_12,-4.282479,POLYGON ((-58.4841308103836 -34.63251938873117...,Se ahorran en total 04:16 minutos al centro
7,10_18_6,-1.999146,POLYGON ((-58.48271948045223 -34.6337255626367...,Se ahorran en total 01:59 minutos al centro
8,10_18_7,-2.332479,POLYGON ((-58.48358552640425 -34.6348253467823...,Se ahorran en total 02:19 minutos al centro
9,10_18_9,-2.215812,POLYGON ((-58.48233821111224 -34.6345238614986...,Se ahorran en total 02:12 minutos al centro


## Home to bikes

In [38]:
#lineas de home to bike con popup de tiempo de recorrido
homesToBikesInsumo = gpd.read_file('networkAnalysisResults/homeToDowntown/homeToDowntown_homeToDowntown.shp')
homesToBikesInsumo = homesToBikesInsumo.to_crs(epsg=4326)

homesToBikesInsumo['CO_FRAC_RA'] = homesToBikesInsumo['Name'].map(lambda x: (x.split()[0]))
homesToBikesInsumo = homesToBikesInsumo.loc[homesToBikesInsumo.CO_FRAC_RA.isin(justImprovement.CO_FRAC_RA),['Name','FacilityID',
                                                                                          'geometry',
                                                                                         'homTbikeTi',
                                                                                          'bikeTsubTi','CO_FRAC_RA']]

homesToBikesInsumo['Name'] = [homesToBikesInsumo.CO_FRAC_RA.loc[i] + ' - ' + str(homesToBikesInsumo.FacilityID.loc[i]) for i in homesToBikesInsumo.index]
homesToBikesInsumo = homesToBikesInsumo.loc[:,['Name','homTbikeTi']]
homesToBikesInsumo.head()

,Name,homTbikeTi
3,10_12_9 - 1008,3.383333
5,10_13_12 - 1008,2.100000
7,10_18_6 - 1008,1.216667
8,10_18_7 - 1008,3.750000
9,10_18_9 - 1008,6.166667


In [45]:
homesToBikes = gpd.read_file('networkAnalysisResults/homeToBikesQGIS/homeToBikes.shp')
homesToBikes = homesToBikes.to_crs(epsg=4326)
homesToBikes = caca.loc[:,['Name','geometry']]
homesToBikes = homesToBikes.merge(homesToBikesInsumo,on='Name',how='inner')
homesToBikes['popupContent'] = homesToBikes.homTbikeTi.map(lambda x: "Son %02d:%02d minutos caminando hasta la nueva estación de bici" % (abs(int(x)),abs((x-int(x)))*60))
homesToBikes.drop(['homTbikeTi'],axis=1,inplace=True)
homesToBikes.to_file('../homesToBikes', driver="GeoJSON")

homesToBikes.head()

,Name,geometry,popupContent
0,10_12_9 - 1008,LINESTRING (-58.48185769645809 -34.63260753795...,Son 03:22 minutos caminando hasta la nueva est...
1,10_13_12 - 1008,LINESTRING (-58.48425299483299 -34.63339834844...,Son 02:06 minutos caminando hasta la nueva est...
2,10_18_6 - 1008,LINESTRING (-58.48252844639142 -34.63412616869...,Son 01:13 minutos caminando hasta la nueva est...
3,10_18_7 - 1008,LINESTRING (-58.48400791416449 -34.63564448277...,Son 03:45 minutos caminando hasta la nueva est...
4,10_18_9 - 1008,LINESTRING (-58.48123267880796 -34.63568356898...,Son 06:10 minutos caminando hasta la nueva est...


## Bike to subway

In [76]:
#lineas de bike to subway con popup de tiempo de recorrido
bikeToSubway = gpd.read_file('networkAnalysisResults/bikeToSubwayQGIS/bikeToSubway_routes.shp')
#10km/h 10 is the velocity in kilometers
bikeToSubway['bikeTsubTime']= bikeToSubway.Total_Leng/1000/10*60 

bikeToSubway['popupContent'] = bikeToSubway.bikeTsubTime.map(lambda x: "Son %02d:%02d minutos de recorrido en bici al subte" % (abs(int(x)),abs((x-int(x)))*60))
bikeToSubway.drop(['FacilityCu','FacilityID','bikeTsubTime','FacilityRa','IncidentCu','IncidentID','ObjectID','Total_Leng'],
                 axis = 1,
                 inplace = True)

bikeToSubway.to_file('../bikeToSubway', driver="GeoJSON")

bikeToSubway.head()

,Name,geometry,popupContent
0,83 - 1008,LINESTRING (-58.46963912075837 -34.63070693207...,Son 08:01 minutos de recorrido en bici al subte
1,83 - 1455,LINESTRING (-58.46963912075837 -34.63070693207...,Son 12:37 minutos de recorrido en bici al subte
2,81 - 1506,LINESTRING (-58.48638523148121 -34.57431885209...,Son 08:55 minutos de recorrido en bici al subte
3,67 - 1818,LINESTRING (-58.46113147747113 -34.64300625780...,Son 13:52 minutos de recorrido en bici al subte
4,71 - 1970,LINESTRING (-58.46238146692324 -34.55564372444...,Son 11:16 minutos de recorrido en bici al subte


In [83]:
#estaciones de subte con estacion en popupcontent
subwayStations = gpd.read_file('../originalShapes/subwayStations/estaciones_de_subte.shp')
subwayStations = subwayStations.to_crs(epsg=4326)
subwayStations.drop(['ID','LINEA'],axis=1,inplace=True)
subwayStations.columns=['popupContent','geometry'] 
bikeToSubway.to_file('../bikeToSubway', driver="GeoJSON")

subwayStations.head()



,popupContent,geometry
0,CASEROS,POINT (-58.39833005827354 -34.63619629114226)
1,INCLAN,POINT (-58.40037199947714 -34.62982192437689)
2,HUMBERTO 1°,POINT (-58.40172516912597 -34.62353873781329)
3,VENEZUELA,POINT (-58.40413412280196 -34.61568876109078)
4,ONCE - 30 DE DICIEMBRE,POINT (-58.40543878453062 -34.60938200117359)


In [89]:
#lineas de subte con popup la inea
subwayLines = gpd.read_file('../originalShapes/subwayLines/red_de_subte.shp')
subwayLines = subwayLines.to_crs(epsg=4326)
subwayLines.drop(['ID'],axis=1,inplace=True)
subwayLines.columns=['popupContent','geometry'] 
subwayLines.head()

,popupContent,geometry
0,LINEA D,LINESTRING (-58.45152661752446 -34.56666320460...
1,LINEA D,LINESTRING (-58.45589001661655 -34.56275716083...
2,LINEA D,LINESTRING (-58.44406940881534 -34.57046014797...
3,LINEA D,LINESTRING (-58.43441511235061 -34.57562603880...
4,LINEA D,LINESTRING (-58.42511333936363 -34.57886957667...


In [92]:
#estaciones viejas con popup nombre la escacion
oldStations = gpd.read_file('../originalShapes/bikeshareStations/estaciones_de_bicicletas.shp')
oldStations = oldStations.to_crs(epsg=4326)
oldStations.drop(['AUTOMAT','DOMICILIO','Horario','IMAGEN','NOMBRE','NRO_EST','OBSERV'],axis=1,inplace=True)
oldStations.columns=['popupContent','geometry'] 
oldStations.head()

,popupContent,geometry
0,"FIGUEROA ALCORTA, PRES. AV. y BIBILONI, JUAN A.",POINT (-58.39185516097447 -34.58358035902182)
1,"DEL LIBERTADOR AV. y RAMOS MEJIA, JOSE MARIA, ...",POINT (-58.37422545676812 -34.59303579060376)
2,"HUERGO, ING. AV. y BELGRANO AV.",POINT (-58.36832159551064 -34.61168855013559)
3,LAVALLE y BOUCHARD,POINT (-58.36835420317743 -34.60216783080737)
4,"GARCIA, MARTIN AV. e IRALA",POINT (-58.36876396832855 -34.62839319029082)


In [5]:
#nueva estaciones
newStations = gpd.read_file('networkAnalysisResults/locationAllocation/bikesOutputLocationAllocation.shp')
newStations = newStations.to_crs(epsg=4326)
newStations = newStations.loc[:,['bikesID','geometry']]
newStations.to_file('../newStations', driver="GeoJSON")

newStations.head()

,bikesID,geometry
0,1008,POINT (-58.48271903122684 -34.63372489684256)
1,1455,POINT (-58.47717057647716 -34.61421608467159)
2,1506,POINT (-58.47657764368105 -34.56641627994559)
3,1818,POINT (-58.47304090788498 -34.65855188127559)
4,1970,POINT (-58.4713938756251 -34.542134330909)
